# Curso de Classificação: 
## aprendendo a classificar dados com Machine Learning

In [73]:
import pandas as pd

import plotly.express as px

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


### Importando dados

In [2]:
df = pd.read_csv('/home/franciscofoz/Documents/GitHub/machine-learning-training/Datasets/marketing_investimento.csv')
df

,idade,estado_civil,escolaridade,inadimplencia,saldo,fez_emprestimo,tempo_ult_contato,numero_contatos,aderencia_investimento
0,45,casado (a),superior,nao,242,nao,587,1,sim
1,42,casado (a),medio,nao,1289,nao,250,4,sim
2,23,solteiro (a),superior,nao,363,nao,16,18,nao
3,58,divorciado (a),superior,nao,1382,nao,700,1,sim
4,50,casado (a),medio,nao,3357,nao,239,4,sim
...,...,...,...,...,...,...,...,...,...
1263,52,solteiro (a),superior,nao,83,nao,1223,6,sim
1264,35,solteiro (a),superior,nao,5958,nao,215,1,sim
1265,30,solteiro (a),superior,nao,-477,sim,1532,2,sim
1266,42,casado (a),superior,nao,2187,nao,525,3,sim


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   idade                   1268 non-null   int64 
 1   estado_civil            1268 non-null   object
 2   escolaridade            1268 non-null   object
 3   inadimplencia           1268 non-null   object
 4   saldo                   1268 non-null   int64 
 5   fez_emprestimo          1268 non-null   object
 6   tempo_ult_contato       1268 non-null   int64 
 7   numero_contatos         1268 non-null   int64 
 8   aderencia_investimento  1268 non-null   object
dtypes: int64(4), object(5)
memory usage: 89.3+ KB


### Análise exploratória

In [4]:
print(df['aderencia_investimento'].value_counts(normalize=True))

px.histogram(df,x='aderencia_investimento',text_auto=True,
             width=500,
             title='Aderência investimento')



nao    0.604101
sim    0.395899
Name: aderencia_investimento, dtype: float64


In [5]:
print(df[['estado_civil','aderencia_investimento']].value_counts(normalize=True))

px.histogram(df,x='estado_civil',
             text_auto=True,
             width=500,
             color='aderencia_investimento',
             barmode='group',
             title='Estado civil')


estado_civil    aderencia_investimento
casado (a)      nao                       0.374606
                sim                       0.207413
solteiro (a)    nao                       0.160095
                sim                       0.130126
divorciado (a)  nao                       0.069401
                sim                       0.058360
dtype: float64


In [6]:
print(df[['escolaridade','aderencia_investimento']].value_counts(normalize=True))

px.histogram(df,x='escolaridade',
             text_auto=True,
             width=500,
             color='aderencia_investimento',
             barmode='group',
             title='Escolaridade')

escolaridade  aderencia_investimento
medio         nao                       0.328864
              sim                       0.193218
superior      nao                       0.173502
              sim                       0.152208
fundamental   nao                       0.101735
              sim                       0.050473
dtype: float64


In [7]:
print(df[['inadimplencia','aderencia_investimento']].value_counts(normalize=True))

px.histogram(df,x='inadimplencia',
             text_auto=True,
             width=500,
             color='aderencia_investimento',
             barmode='group',
             title='Inadimplência')

inadimplencia  aderencia_investimento
nao            nao                       0.592271
               sim                       0.389590
sim            nao                       0.011830
               sim                       0.006309
dtype: float64


In [8]:
print(df[['fez_emprestimo','aderencia_investimento']].value_counts(normalize=True))

px.histogram(df,x='fez_emprestimo',
             text_auto=True,
             width=500,
             color='aderencia_investimento',
             barmode='group',
             title='Fez empréstimo?')

fez_emprestimo  aderencia_investimento
nao             nao                       0.503943
                sim                       0.361987
sim             nao                       0.100158
                sim                       0.033912
dtype: float64


In [9]:
px.box(df,y='idade',
             width=500,
             color='aderencia_investimento',
             title='Idade')

In [10]:
px.box(df,y='saldo',
             width=500,
             color='aderencia_investimento',
             title='Saldo')

In [11]:
px.box(df,y='tempo_ult_contato',
             width=500,
             color='aderencia_investimento',
             title='Tempo do último contato')

In [12]:
px.box(df,y='numero_contatos',
             width=500,
             color='aderencia_investimento',
             title='Número de contatos')

### Feature engineering

In [40]:
x = df.drop('aderencia_investimento',axis=1)
y = df['aderencia_investimento']

In [41]:
columns = x.columns

one_hot = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary'),['estado_civil','escolaridade','inadimplencia','fez_emprestimo']),
    remainder='passthrough',sparse_threshold=0
    )

x = one_hot.fit_transform(x)
x = pd.DataFrame(x,columns=one_hot.get_feature_names_out(columns))

In [42]:
y = LabelEncoder().fit_transform(y)

### Divisão de treino e teste

In [69]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y,stratify=y,random_state=5)

### Modelo "base"

In [50]:
dummy = DummyClassifier()
dummy.fit(x_treino,y_treino)

dummy.score(x_teste,y_teste)

0.6025236593059937

### Teste do modelo

#### Decision Tree

In [57]:
arvore = DecisionTreeClassifier(random_state=5,max_depth=10)
arvore.fit(x_treino,y_treino)

arvore.score(x_teste,y_teste)

0.6876971608832808

In [68]:
arvore = DecisionTreeClassifier(random_state=5,max_depth=5)
arvore.fit(x_treino,y_treino)

arvore.score(x_teste,y_teste)

0.722397476340694

#### KNN

In [75]:
x_treino_normalizado = MinMaxScaler().fit_transform(x_treino)
x_teste_normalizado = MinMaxScaler().fit_transform(x_teste)

In [77]:
knn = KNeighborsClassifier()

knn.fit(x_treino_normalizado,y_treino)

knn.score(x_teste_normalizado,y_teste)

0.6750788643533123

## Escolhendo o melhor modelo

In [78]:
print(f'Acurácia Dummy: {dummy.score(x_teste, y_teste)}')
print(f'Acurácia Árvore: {arvore.score(x_teste, y_teste)}')
print(f'Acurácia KNN: {knn.score(x_teste_normalizado, y_teste)}')


Acurácia Dummy: 0.6025236593059937
Acurácia Árvore: 0.722397476340694
Acurácia KNN: 0.6750788643533123
